In [1]:
# imports
from itertools import islice
from riotwatcher import LolWatcher, ApiError
import pandas as pd
import pprint
from riotwatcher import LolWatcher, ApiError
my_region = 'na1'
from api_key import API_KEY

lol_watcher = LolWatcher(API_KEY)

In [2]:
# select my riot puuid
my_region = 'na1'
me = lol_watcher.summoner.by_name(my_region, 'slayer56429')
puid = me["puuid"]
print(puid)

XixSHIdrUeBgVQHvWxw8nXK3yY_ybNbj5oeR36b5Db4T93o35j1bJ1sxgO1l1QW4DDHJyDTSnL4mjQ


In [3]:
# find last 30 ranked games - start time currently indicates latest season at the end of 2022
match_hist = lol_watcher.match.matchlist_by_puuid(my_region,puid,count=30, queue = 420,start_time = 1668582000)

In [4]:
match_hist

['NA1_4494220264',
 'NA1_4494193396',
 'NA1_4494184988',
 'NA1_4494161766',
 'NA1_4494130742']

In [5]:
# pretty print last game data
last = lol_watcher.match.by_id(my_region,match_hist[0])
# all participant data
participants = last["info"]["participants"]

part_num = 0
for p in participants:
    print(f"---------------------------------------Participant # {part_num}--------------------------------------------")
    part_num+=1
    pprint.pprint(p)
# team data 
teams = last["info"]["teams"]
for t in teams:
    pprint.pprint(t)    

---------------------------------------Participant # 0--------------------------------------------
{'allInPings': 0,
 'assistMePings': 5,
 'assists': 2,
 'baitPings': 0,
 'baronKills': 1,
 'basicPings': 0,
 'bountyLevel': 1,
 'challenges': {'12AssistStreakCount': 0,
                'abilityUses': 125,
                'acesBefore15Minutes': 0,
                'alliedJungleMonsterKills': 0,
                'baronBuffGoldAdvantageOverThreshold': 1,
                'baronTakedowns': 1,
                'blastConeOppositeOpponentCount': 0,
                'bountyGold': 0,
                'buffsStolen': 0,
                'completeSupportQuestInTime': 0,
                'controlWardTimeCoverageInRiverOrEnemyHalf': 0.08514242129717947,
                'controlWardsPlaced': 1,
                'damagePerMinute': 905.9239685951351,
                'damageTakenOnTeamPercentage': 0.1753192980194478,
                'dancedWithRiftHerald': 0,
                'deathsByEnemyChamps': 2,
               

In [6]:
# define pull current season function which is set to grab the my last 30 games of the newest season which started on Sep 16 2022
current_season_start = []
runes = []

def pull_current_season():

    current_season_start = []
    runes = []

    for m in range(len(match_hist)):
        game = lol_watcher.match.by_id(my_region,match_hist[m])

        for g in game["info"]["participants"]:
            runes.append(g.pop('perks', None))
            if puid in g['puuid']:
                self_game_data = g
                current_season_start.append(self_game_data)

        current_season_df = pd.DataFrame(current_season_start)
        current_season_df = current_season_df.drop(columns=['riotIdName','riotIdTagline','challenges','puuid','summonerId','summonerName'],axis=1)
        current_season_df.to_csv('current_season_first_30.csv')

In [7]:
pull_current_season()

In [8]:
test = pd.read_csv("current_season_first_30.csv")
# pd.io.sql.get_schema(test,"self_data")
pd.set_option('display.max_rows', None)
test = test.drop(test.columns[0], axis=1)
test.to_csv('current_season_first_30.csv',index=False)
test

,allInPings,assistMePings,assists,baitPings,baronKills,basicPings,bountyLevel,champExperience,champLevel,championId,...,turretKills,turretTakedowns,turretsLost,unrealKills,visionClearedPings,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win
0,1,8,4,0,0,0,0,14138,15,154,...,0,2,2,0,0,23,2,3,2,True
1,3,13,2,0,0,0,4,8071,11,82,...,0,2,0,0,0,13,2,1,3,True
2,0,1,1,0,0,0,3,7382,11,82,...,2,2,0,0,0,11,2,1,2,True
3,2,5,3,0,0,0,0,9243,12,203,...,0,0,5,0,0,22,3,4,3,False
4,2,8,12,0,0,0,0,18660,18,82,...,0,2,4,0,0,49,8,5,5,True
